<a href="https://colab.research.google.com/github/JaeHeee/Pytorch_Tutorial/blob/main/code/NEURAL_NETWORKS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NEURAL NETWORKS

Neural networks은 `torch.nn` package를 사용하여 생성

`nn`은 모델을 정의하고 미분하는데 `autograd`를 사용. `nn.Module`은 layer와 `output`을 반환하는 `forward(input)` 메서드를 포함

Neural networks의 일반적인 train 과정

- 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의

- 데이터셋(dataset) 입력을 반복

- 입력을 신경망에서 전파(process)

- 손실(loss; 출력이 정답으로부터 얼마나 떨어져있는지)을 계산

- 변화도(gradient)를 신경망의 매개변수들에 역으로 전파

- 신경망의 가중치를 갱신. 일반적으로 다음과 같은 간단한 규칙을 사용 :  `새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)`

### Define the network



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # affine : y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # (2, 2) 크기 윈도우에 대해 맥스 풀링(max pooling)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # 크기가 제곱수라면 하나의 숫자만을 특정
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)    

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


`forward` 함수만 정의하고 나면, `backward` 함수는 `autograd` 를 사용하여 자동으로 정의한다. `forward` 함수에서는 어떠한 Tensor 연산을 사용가능

모델의 학습 가능한 매개변수들은 `net.parameters()` 에 의해 반환

In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


임의의 32x32 input 넣어보기

In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0181, -0.0152, -0.0927,  0.0735,  0.1141, -0.0760, -0.1455, -0.0771,
          0.0546, -0.0692]], grad_fn=<AddmmBackward>)


모든 매개변수의 gradient buffer를 0으로 설정하고, 무작위 값으로 backprop

In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

`torch.nn` 은 mini-batch만 지원. `torch.nn` 패키지 전체는 하나의 샘플이 아닌, 샘플들의 mini-batch만을 입력으로 받는다.

예를 들어, `nn.Conv2D` 는 `nSamples x nChannels x Height x Width` 의 4차원 Tensor를 입력으로 한다.

만약 하나의 샘플만 있다면, `input.unsqueeze(0)` 을 사용해서 가상의 차원을 추가

#### summary

- `torch.Tensor` - `backward()` 같은 `autograd` 연산을 지원하는 다차원 배열. 또한 tensor에 대한 gradient를 갖고 있다.

- `nn.Module` - 신경망 모듈. 매개변수를 encapsulation하는 간편한 방법 으로, GPU로 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공.

- `nn.Parameter` - Tensor의 한 종류로, Module 에 속성으로 할당될 때 자동으로 매개변수로 등록

- `autograd.Function` - autograd 연산의 전방향과 역방향 정의 를 구현. 모든 Tensor 연산은 하나 이상의 `Function` 노드를 생성하며, 각 노드는 Tensor 를 생성하고 이력(history)을 부호화 하는 함수들과 연결

### Loss Function

손실 함수는 (output, target)을 한 쌍(pair)의 입력으로 받아, 출력(output)이 정답(target)으로부터 얼마나 멀리 떨어져있는지 추정하는 값을 계산

nn 패키지에는 여러가지의 손실 함수들 이 존재합니다. 간단한 손실 함수로는 출력과 대상간의 평균제곱오차(mean-squared error)를 계산하는 `nn.MSEloss` 가 있다.

In [ ]:
output = net(input)
target = torch.randn(10)
print(target)
print(target.size())

tensor([ 1.8679, -1.6043,  1.4283,  0.9151, -0.8611,  1.8825, -0.0534,  1.0908,
        -0.0666,  1.0313])
torch.Size([10])


In [ ]:
target = target.view(1, -1)
print(target)
print(target.size())

tensor([[ 1.8679, -1.6043,  1.4283,  0.9151, -0.8611,  1.8825, -0.0534,  1.0908,
         -0.0666,  1.0313]])
torch.Size([1, 10])


In [ ]:
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.6489, grad_fn=<MseLossBackward>)


`.grad_fn` 속성을 사용하여 `loss` 를 역방향에서 따라가다보면, 이러한 모습의 연산 그래프를 볼 수 있다

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  
      -> view -> linear -> relu -> linear -> relu -> linear  
      -> MSELoss  
      -> loss  

따라서 `loss.backward()` 를 실행할 때, 전체 그래프는 손실(loss)에 대하여 미분되며, 그래프 내의 `requires_grad=True` 인 모든 Tensor는 변화도(gradient)가 누적된 `.grad` Tensor를 갖게된다.

In [ ]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

### Backprop

error를 backprop하기 위해서는 `loss.backward()` 만 해주면 된다. 기존 gradient를 없애는 작업이 필요한데, 그렇지 않으면 gradient가 기존의 것에 누적되기 때문입니다.

In [ ]:
net.zero_grad() # 모든 매개변수의 gradient buffer를 0으로 만듦

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0097, -0.0041,  0.0181, -0.0112,  0.0006, -0.0036])


[deep neural network를 구성하는 다양한 module과 loss function이 포함되어 있는 nn package](https://pytorch.org/docs/stable/nn.html)

### Update the weights

실제로 많이 사용되는 가장 단순한 갱신 규칙은 SGD; Stochastic Gradient Descent  
`새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)`



In [ ]:
# 간단한 python code
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

neural network를 구성할 때 SGD, Nesterov-SGD, Adam, RMSProp 등과 같은 다양한 갱신 규칙을 사용하고 싶을 수 있다. 이를 위해서 `torch.optim` 라는 작은 패키지에 이러한 방법들을 모두 구현되어 있다.

In [ ]:
import torch.optim as optim

# Optimizer를 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in training loop
optimizer.zero_grad() # gradient buffer를 0으로
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

`optimizer.zero_grad()` 를 사용하여 수동으로 변화도 버퍼를 0으로 설정하는 것은 gradient가 누적되기 때문